# Test Homogeneous Poisson

In [1]:
# std library
import os
import sys
import json
import random
from datetime import datetime

# third-party
import numpy as np
from scipy.interpolate import interp1d
from matplotlib import pyplot as plt
import cmdstanpy
cmdstanpy.install_cmdstan()
from cmdstanpy import CmdStanModel

CmdStan install directory: /Users/zhuanglinsheng/.cmdstan
CmdStan version 2.36.0 already installed
Test model compilation


In [2]:
wd = os.getcwd()
current_dir = os.path.dirname(wd + '/')
parent_dir = os.path.dirname(current_dir)

# wd, current_dir, parent_dir
sys.path.append(parent_dir)

# import local modual
from poisson import simulate_hpp

In [3]:
start_time = datetime(2025, 1, 1, 0, 0, 0)
end_time = datetime(2025, 4, 1, 0, 0, 0)

In [4]:
# Set parameter
real_arrival_rate = 0.1

In [5]:
# Generate data
rng_poisson = random.Random(42)  # set seed for generator
events = simulate_hpp(start_time, end_time, real_arrival_rate, rng_poisson)

# Save the data to json
test_poisson_data = os.path.join(wd, 'test_poisson_data.json')
with open(test_poisson_data, 'w') as f:
	json.dump({
		'T': (end_time - start_time).total_seconds() / 3600,
		'N': len(events),
		'events': [(t - start_time).total_seconds() / 3600 for t in events],
	}, f, indent=4)

In [6]:
# Build model
stan_file_poisson = os.path.join(wd, 'test_poisson.stan')
stan_model_poisson = CmdStanModel(stan_file=stan_file_poisson)

In [7]:
# Fit data
fit_poisson = stan_model_poisson.sample(
	data=test_poisson_data,
	chains=4,
	iter_warmup=2000,
	iter_sampling=4000,
	parallel_chains=4,
	show_console=False,
)

16:55:30 - cmdstanpy - INFO - CmdStan start processing


chain 1 |          | 00:00 Status

chain 2 |          | 00:00 Status

chain 3 |          | 00:00 Status

chain 4 |          | 00:00 Status

16:55:30 - cmdstanpy - INFO - CmdStan done processing.


Compare:
- real_arrival_rate = 0.1

In [8]:
fit_poisson.summary()

,Mean,MCSE,StdDev,MAD,5%,50%,95%,ESS_bulk,ESS_tail,R_hat
lp__,-704.793000,0.007550,0.699375,0.312829,-706.198000,-704.52500,-704.297000,8305.51,11041.20,1.00022
lambda,0.098157,0.000087,0.006723,0.006781,0.087416,0.09809,0.109461,5943.46,9931.66,1.00077
